# Feature Research

Explore and validate technical indicators and feature engineering for trading strategies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

from src.features.indicators import atr, rsi, zscore, vwap, bollinger_bands, macd, ema
from src.features.feature_sets import compute_base_features

# Load sample data
df = pd.read_csv('data/sample/eod_reliance.csv', parse_dates=['date'])
df['symbol'] = 'RELIANCE'
df['datetime'] = df['date']
print(f'Loaded {len(df)} rows')
df.head()

In [ ]:
# Compute all features
featured = compute_base_features(df)
print(f'Features added: {len(featured.columns) - len(df.columns)}')
print(f'\nFeature columns:')
new_cols = [c for c in featured.columns if c not in df.columns]
for c in new_cols:
    print(f'  {c}')

In [ ]:
# RSI analysis
rsi_vals = rsi(df['close'], period=14)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df['date'], df['close'], label='Close', color='steelblue')
axes[0].set_ylabel('Price')
axes[0].set_title('RELIANCE Price + RSI')

axes[1].plot(df['date'], rsi_vals, label='RSI(14)', color='purple')
axes[1].axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought')
axes[1].axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold')
axes[1].fill_between(df['date'], 30, 70, alpha=0.1, color='gray')
axes[1].set_ylabel('RSI')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Bollinger Bands
upper, middle, lower = bollinger_bands(df['close'], window=20, num_std=2.0)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df['date'], df['close'], label='Close', color='steelblue')
ax.plot(df['date'], upper, '--', color='red', alpha=0.5, label='Upper BB')
ax.plot(df['date'], middle, '--', color='gray', alpha=0.5, label='SMA(20)')
ax.plot(df['date'], lower, '--', color='green', alpha=0.5, label='Lower BB')
ax.fill_between(df['date'], lower, upper, alpha=0.1, color='gray')
ax.set_title('Bollinger Bands')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Feature correlations
feature_cols = [c for c in featured.columns if c not in ['date', 'datetime', 'symbol']]
corr = featured[feature_cols].corr()

fig, ax = plt.subplots(figsize=(16, 12))
im = ax.imshow(corr.values, cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xticks(range(len(feature_cols)))
ax.set_yticks(range(len(feature_cols)))
ax.set_xticklabels(feature_cols, rotation=90, fontsize=7)
ax.set_yticklabels(feature_cols, fontsize=7)
plt.colorbar(im)
ax.set_title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Z-score for mean reversion signals
zs = zscore(df['close'], window=20)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
axes[0].plot(df['date'], df['close'], color='steelblue')
axes[0].set_ylabel('Price')
axes[0].set_title('Price + Z-Score Signals')

axes[1].plot(df['date'], zs, color='purple')
axes[1].axhline(y=2, color='red', linestyle='--', alpha=0.5, label='Short entry (z>2)')
axes[1].axhline(y=-2, color='green', linestyle='--', alpha=0.5, label='Long entry (z<-2)')
axes[1].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
axes[1].fill_between(df['date'], -2, 2, alpha=0.05, color='gray')
axes[1].set_ylabel('Z-Score')
axes[1].legend()

plt.tight_layout()
plt.show()